# Risk Parity Portfolio

For this project, I am replicating the results from the 2012 paper, "Leverage Aversion and Risk Parity" by Asness, Frazzini, and Pedersen. This paper compares the stock portfolio, bond portfolio, value-weighted stock/bond portfolio, 60/40 portfolio, risk parity unlevered portfolio, risk parity portfolio, risk parity minus value-weighted portfolio and risk parity minus 60/40 portfolio. The output is from January 1929 to June 2010, at a monthly frequency.

Same as the Historical Market Return project, I imported the CRSP data for stocks and the risk free rate. I also use CRSP for the bond pricing as well. I combine the dlret and ret data for the stocks and encompass all returns in the final 'ret' column for stocks. In accordance with the French Fama methods, I use the 10 and 11 sharecodes and the 1, 2 and 3 exchanges. The URL containing details on this method is below:

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Below is a list of relevant libraries used in this project.

In [18]:
import wrds
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pandas_datareader
import datetime
import numpy as np

Below I connected the script to the WRDS servers so I can access the data for the stocks, delisted stocks, bonds and risk free rate.

In [19]:
id = 'johncrowe'
conn = wrds.Connection(id)

# Download monthly crsp stock data
a = conn.raw_sql("""
                    select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                    a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr
                    from crspq.msf as a 
                    left join crsp.msenames as b
                    on a.permno=b.permno
                    and b.namedt<=a.date
                    and a.date<=b.nameendt
                    where a.date between '01/01/1900' and '12/31/2023'
                    """)

# Download monthly crsp delisted stock data
b = conn.raw_sql("""
                    select permno, dlret, dlstdt, dlstcd
                    from crspq.msedelist
                    """)

# Download bond data
c = conn.raw_sql("""
                    select kycrspid, mcaldt, tmretnua, tmtotout
                    from crspq.tfz_mth
                    """)

# Download risk free data
d = conn.raw_sql("""
                    select caldt, t30ret, t90ret
                    from crspq.mcti
                    """)

conn.close()

Enter your WRDS username [John]:johncrowe
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


Below we create new dataframes so that we do not have to reconnect to the SQL servers to rerun our code.

In [20]:
crsp_raw = a.copy()
dlret_raw = b.copy()
bonds_raw = c.copy()
french = d.copy()

Below we are cleaning the monthly risk free rate.

In [21]:
# Rename French Fama data to more useable format
french = french.rename(columns={"caldt":"date","t30ret":"rf30"}).copy()

# Convert columns to correct datatypes
french['date'] = pd.to_datetime(french['date']) + MonthEnd(0)
french['rf30'] = french['rf30'].astype(float)

# Create year and month columns for french data
french['year'] = french['date'].dt.year
french['month'] = french['date'].dt.month

# Keep necessary columns and drop NAs
french = french[['year','month', 'rf30']]
french = french.dropna()

Here we use some data cleaning and rearranging for our CRSP stocks and delisted stocks. Same as in our Historical Market Return project, I arrange by date and permno. I then manipulate the date portion so that I can ensure they are in the proper order and then merge delisted with crsp to create our total return column, which I will call 'ret'. I then attach the lagged market equity and end up with a row in our dataframe for each stock-year-month. We are not yet factoring in the risk free rate to determine excess returns.

In [22]:
# Clean crsp_raw data
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop=True).copy()
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%y-%m-%d') + MonthEnd(0)
crsp_raw['prc'] = np.absolute(crsp_raw['prc'])

# Clean dlret_raw data
dlret_raw = dlret_raw.sort_values(['permno', 'dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date'] = dlret_raw['dlstdt'] + MonthEnd(0)

# Merge crsp_raw with dlret_raw
stocks = crsp_raw.merge(dlret_raw[['permno','date','dlret']], how='outer', on=['permno','date'])
stocks['dlret'] = stocks['dlret'].fillna(0)
stocks['shrcd'] = stocks['shrcd'].ffill()
stocks['exchcd'] = stocks['exchcd'].ffill()
stocks['ret'] = stocks['ret'].fillna(0)
stocks['prc'] = stocks['prc'].fillna(0)
stocks['shrout'] = stocks['shrout'].fillna(0)
stocks['me'] = stocks['prc']*stocks['shrout']

# Redefine 'ret' to include delisted return
stocks['ret'] = (1 + stocks['ret']) * (1 + stocks['dlret']) - 1

# Attach Lagged Market Equity (to be used as weights) for each stock-year-month
stocks = stocks.sort_values(by=['permno','date']).reset_index().drop('index',axis=1).copy()
stocks['daten'] = stocks['date'].dt.year*12 + stocks['date'].dt.month
stocks['IsValidLag'] = stocks['daten'].diff(1) == 1 # Lag date has to be the lagged date
stocks.loc[stocks[stocks['permno'].diff(1) != 0].index,['IsValidLag']] = False # Lagged date has to be the same security
stocks['lme'] = stocks[['permno','me']].groupby('permno').shift(1)
stocks.loc[stocks[stocks['IsValidLag'] == False].index,['lme']] = np.nan
stocks = stocks.drop(['IsValidLag','daten'], axis=1)

# Filter for 'shrcd' = 10 or 11 and 'exchcd' = 1, 2 or 3 per the Fama French website
stocks = stocks[((stocks['shrcd'] == 10) | (stocks['shrcd'] == 11)) & ((stocks['exchcd'] == 1) | (stocks['exchcd'] == 2) | (stocks['exchcd'] == 3))].copy()

# Create year and month columns for readability
stocks['year'] = stocks['date'].dt.year
stocks['month'] = stocks['date'].dt.month

# Rearrange columns into desired format
stocks = stocks[['permno', 'year', 'month', 'prc', 'shrout', 'lme', 'ret']]

# Drop NAs so that columns without lagged market equity value are not included
stocks = stocks.dropna()

With our stocks set up in a stock-year-month format, we now want to do the same with out bonds, attaching the lagged market equity column. The bond 'me' column is in millions, so we want to multiply by 1,000 to match our stock lagged market equity format. We are not yet factoring in the risk free rate to determine excess returns.

In [23]:
# Clean bonds_raw and convert columns to proper datatypes
bonds_raw = bonds_raw.dropna(axis=0, how='any')
bonds_raw['mcaldt'] = pd.to_datetime(bonds_raw['mcaldt'], format='%y-%m-%d') + MonthEnd(0)
bonds_raw = bonds_raw.rename(columns={"mcaldt":"date","tmretnua":"ret","tmtotout":"me","kycrspid":"idCRSP"}).copy()
bonds_raw['ret'] = bonds_raw['ret'].astype(float)
bonds_raw['me'] = bonds_raw['me'].astype(float)
bonds_raw['idCRSP'] = bonds_raw['idCRSP'].astype(str)

# Attach Lagged Market Equity (to be used as weights) for each stock-year-month
bonds_raw['daten'] = bonds_raw['date'].dt.year*12 + bonds_raw['date'].dt.month
bonds_raw_lagged = bonds_raw.copy()
bonds_raw_lagged['daten'] = bonds_raw_lagged['daten'] + 1
bonds_raw_lagged = bonds_raw_lagged.rename(columns={"me":"lme"}).copy()

# Add dataframes for first month not captured in our inner merge method and merge 
min_daten = min(bonds_raw_lagged['daten'])
first_month = bonds_raw_lagged[bonds_raw_lagged['daten'] == min_daten].copy()
first_month['lme'] = first_month['lme']/(1 + first_month['ret'])
first_month['daten'] = first_month['daten'] - 1
bonds_raw_lagged = pd.concat([first_month, bonds_raw_lagged])
bonds_raw_lagged = bonds_raw_lagged[['idCRSP', 'daten', 'lme']]
bonds = bonds_raw.merge(bonds_raw_lagged, how='inner', on=['daten', 'idCRSP'])

# Create year and month columns and rearrange, convert bonds currency from millions to thousands
bonds['year'] = bonds['date'].dt.year
bonds['month'] =  bonds['date'].dt.month
bonds = bonds.sort_values(['idCRSP', 'year', 'month']).reset_index(drop=True).copy()
bonds['lme'] = 1000*bonds['lme']
bonds = bonds[['idCRSP', 'year', 'month', 'lme', 'ret']]

Below we can use our bonds, stocks and risk free rates to find the excess value weighted returns on both stocks and bonds as well as the lagged market value for each month. We will end up with a dataframe corresponding to a row for each year-month combination present in all three dataframes.

There were two different risk free return sets available in our data, one for 30 day and one for 90 day risk free rates. I decided to use the 30 day risk free rate. The main reason for doing this is because we are calculating the monthly returns on the stocks and bonds. Many of these portfolios need to be rebalanced monthly in order to replicate these returns. If I were a portfolio manager, it makes more sense to use the 30 day risk free rate since the portfolio could be changing after 30 days to maintain some of the strategies explored in the "Leverage Aversion and Risk Parity" paper such as the 60-40 strategy or RP strategy. Therefore, we used the 30 day risk free rate to replicate all of these numbers.

In [24]:
def monthly_returns(stocks, bonds, french):

    # Create lists for our final dataframe
    years = []
    months = []
    lagged_market_values = []
    stock_excess_returns = []
    lagged_bond_values = []
    bond_excess_returns = []
    
    # Find all dates present in stocks, bonds and risk free
    stock_dates = stocks[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    bond_dates = bonds[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    french_dates = french[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    date = (stock_dates.merge(bond_dates, how='inner', on=['year', 'month'])).merge(french_dates, how='inner', on=['year', 'month'])
    
    # Iterate through all unique year-month combinations present in our bond data
    for index, row in date.iterrows():
        
        year = row['year']
        month = row['month']
        RF = french[(french['year'] == year) & (french['month'] == month)]['rf30'].iloc[0]
        
        # Get bond market values
        this_bonds = bonds[(bonds['year'] == year) & (bonds['month'] == month)]
        bond_start_value = this_bonds['lme'].sum()
        bond_end_value = (this_bonds['lme'] * (this_bonds['ret'] + 1)).sum()
        bond_excess_return = (bond_end_value - bond_start_value)/bond_start_value - RF
        
        # Get stock market values
        this_stocks = stocks[(stocks['year'] == year) & (stocks['month'] == month)]
        stock_start_value = this_stocks['lme'].sum()
        stock_end_value = (this_stocks['lme'] * (this_stocks['ret'] + 1)).sum()
        stock_excess_return = (stock_end_value - stock_start_value)/stock_start_value - RF
        
        # Append results
        years.append(year)
        months.append(month)
        lagged_market_values.append(stock_start_value)
        stock_excess_returns.append(stock_excess_return)
        lagged_bond_values.append(bond_start_value)
        bond_excess_returns.append(bond_excess_return)
        
        
    # Create a DataFrame using our monthly returns
    data = {'Year': years,
            'Month': months,
            'Stock_lag_MV': lagged_market_values,
            'Stock_Excess_Vw_Ret' : stock_excess_returns,
            'Bond_lag_MV': lagged_bond_values,
            'Bond_Excess_Vw_Ret': bond_excess_returns}
        
    Bond_data = pd.DataFrame(data)
    
    return(Bond_data)




In [25]:
a1 = monthly_returns(stocks, bonds, french)
a1

,Year,Month,Stock_lag_MV,Stock_Excess_Vw_Ret,Bond_lag_MV,Bond_Excess_Vw_Ret
0,1926,1,2.690395e+07,-0.002704,1.936987e+07,0.003870
1,1926,2,2.703235e+07,-0.036744,1.950200e+07,0.001076
2,1926,3,2.616208e+07,-0.067653,1.888400e+07,0.000810
3,1926,4,2.450693e+07,0.033663,1.873600e+07,0.003321
4,1926,5,2.529619e+07,0.011907,1.922700e+07,0.002287
...,...,...,...,...,...,...
1172,2023,9,4.422717e+10,-0.052413,2.063664e+10,-0.021788
1173,2023,10,4.204562e+10,-0.031793,2.079155e+10,-0.016106
1174,2023,11,4.074575e+10,0.088702,2.072124e+10,0.029037
1175,2023,12,4.439292e+10,0.048707,2.105048e+10,0.023462


Next we will calculate the monthly unlevered and levered risk-parity portfolio returns as defined by Asness, Frazzini, and Pedersen (2012). For the levered risk-parity portfolio, we match the value-weighted portfolio’s $\hat{σ}$ over the longest matched holding period of both. The output is from January 1926 to December 2023, at a monthly frequency

We already calculated the bond and stock excess return in questions 1 and 2, so we are familiar with that procedure. The excess value weighted return was calculated by investing in stocks and bonds in proportion to their lagged market value, therefore the exact proportion invested in stocks and bond changed every month depending on the cap relative to both markets. The excess 60-40 return invested 60% in stocks and 40% in bonds, regardless of their respective market caps and is rebalanced monthly to maintain weights.

The stock/bond inverse sigma hat was found by taking the inverse of the rolling 3 year monthly volatility. While there are ways to calculate inverse sigma hat for the first 3 years of data (1926-1928), I thought the results would not be reliable enough for our RP portfolio. Therefore, I decided to use 'NaN' values for the risk parity portfolios for the first 3 years. We can also note that since bonds usually have lower volatility than stocks, the inverse sigma hat of the bonds was typically around 10 times higher than for the stocks. This means the RP portfolios tend to invest much more in the bonds.

The Unlevered K was calculated for each month with the equation below, per the equation in Asness, Frazzini, and Pedersen (2012).

$$
K_{unlevered} = \frac{1}{\hat{\sigma}_{bonds} + \hat{\sigma}_{stocks}}
$$

This equation is used so that we can invest in stocks and bonds based on their volatility (the more volatility, the less we invest). Multiplying $K_{unlevered}$ by $\hat{\sigma}_{bonds}$ and $\hat{\sigma}_{stocks}$ gives the portfolio weights for bonds and stocks, respectively. This is how we created our RP unlevered portfolio so that we can target equal risk allocation across the available instruments with it being rebalanced monthly.

For the RP portfolio, these weights are multiplied by a constant to match the ex post realized volatility of the value-weighted benchmark. This means the RP portfolio will have a constant K every month of the portfolio. Below, we find K to be around 0.020556. This was optimized in the for loop below, which goes through many different proposed values of K and finds which K value creates the RP portfolio with the closest volatility to the value weighted portfolio. Once we had this K, we were able to weigh this portfolio using the same method as for the unlevered portfolio.


In [26]:
def portfolio_returns(df):
    
    # Find VW Stock columns, creating total market value column and stock and bond weights
    df['total_market_value'] = df['Stock_lag_MV'] + df['Bond_lag_MV']
    df['v_weight_stocks'] = df['Stock_lag_MV']/df['total_market_value']
    df['v_weight_bonds'] = df['Bond_lag_MV']/df['total_market_value']
    df['Excess_Vw_Ret'] = df['v_weight_stocks']*df['Stock_Excess_Vw_Ret'] + df['v_weight_bonds']*df['Bond_Excess_Vw_Ret']

    # Create 60-40 returns
    df['Excess_60_40_Ret'] = .6*df['Stock_Excess_Vw_Ret'] + .4*df['Bond_Excess_Vw_Ret']

    # Create stock/bond volatility columns
    df['Stock_volatility'] = df['Stock_Excess_Vw_Ret'].rolling(window=36).std().shift(1)
    df['Bond_volatility'] = df['Bond_Excess_Vw_Ret'].rolling(window=36).std().shift(1)

    # Use volatility column to create inverse sigma hat columns
    df['Stock_inverse_sigma_hat'] = 1/df['Stock_volatility']
    df['Bond_inverse_sigma_hat'] = 1/df['Bond_volatility']

    # Use inverse sigma hat columns to create unlevered_k column
    df['Unlevered_K'] = 1/(df['Stock_inverse_sigma_hat'] + df['Bond_inverse_sigma_hat'])
    
    # Use unlevered_k column to find portfolio weights for unlevered rp
    df['Excess_Unlevered_RP_Ret'] = df['Unlevered_K']*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret'] + df['Unlevered_K']*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret']

    # Find std for Excess VW for our RP levered portfolio
    Excess_Vw_Ret_std = df['Excess_Vw_Ret'].std()
    
    # Now lets find Levered_K, such that our leveraged portfolio is equal to the VW Volatility
    min_diff = 10000
    
    for k_test in range(1,50001):
        
        k_test = k_test/1000000
        Leveraged_Ret = k_test*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret'] + k_test*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret']
        Leveraged_Ret_std = Leveraged_Ret.std()
    
        diff = abs(Leveraged_Ret_std - Excess_Vw_Ret_std)
    
        if (diff<min_diff):
            min_diff = diff
            k = k_test
            
    # Create our levered_k value
    df['Levered_k'] = k
    
    # Use our levered_k value for our levered portfolio weights
    df['Excess_Levered_RP_Ret'] = k*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret'] + k*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret']

    # Drop the columns we created for theses calculations that we will not need
    df.drop(columns=['Stock_lag_MV', 'Bond_lag_MV', 'total_market_value', 'v_weight_stocks', 'v_weight_bonds', 'Stock_volatility', 'Bond_volatility'], inplace=True)
    
    return(df)

In [27]:
a2 = portfolio_returns(a1)
a2

,Year,Month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_inverse_sigma_hat,Bond_inverse_sigma_hat,Unlevered_K,Excess_Unlevered_RP_Ret,Levered_k,Excess_Levered_RP_Ret
0,1926,1,-0.002704,0.003870,0.000048,-0.000074,NaN,NaN,NaN,NaN,0.020557,NaN
1,1926,2,-0.036744,0.001076,-0.020894,-0.021616,NaN,NaN,NaN,NaN,0.020557,NaN
2,1926,3,-0.067653,0.000810,-0.038952,-0.040268,NaN,NaN,NaN,NaN,0.020557,NaN
3,1926,4,0.033663,0.003321,0.020517,0.021526,NaN,NaN,NaN,NaN,0.020557,NaN
4,1926,5,0.011907,0.002287,0.007753,0.008059,NaN,NaN,NaN,NaN,0.020557,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1172,2023,9,-0.052413,-0.021788,-0.042670,-0.040163,19.150241,77.869270,0.010307,-0.027833,0.020557,-0.055511
1173,2023,10,-0.031793,-0.016106,-0.026602,-0.025518,18.985443,76.149721,0.010511,-0.019236,0.020557,-0.037620
1174,2023,11,0.088702,0.029037,0.068588,0.064836,18.915844,75.401337,0.010603,0.041003,0.020557,0.079500
1175,2023,12,0.048707,0.023462,0.040587,0.038609,19.656502,69.560416,0.011209,0.029024,0.020557,0.053231


Next I want to replicate and report Panel A of Table 2 in Asness, Frazzini, and Pedersen (2012) by reporting the annualized average excess returns, t-statistic of the average excess returns, annualized volatility, annualized sharpe ratio, skewness, and excess kurtosis. The sample is from January 1929 to June 2010, at monthly frequency.

Below you can see the annulaized_portfolio_returns function takes the dataframe output from portfolio_returns and calculates the annualized mean, std, etc. using the built in functions on Python. While the replication is not exact, it tends to range from less than 1 basis point off to around 10 basis points off. The difference is not nonzero but can be considered economically negligible in many situations. 

There are several reasons why our replication was not exact. The authors may have employed specific methodologies or adjustments in their calculations that we are not aware of or cannot replicate exactly. For instance, they might have applied certain filters, transformations, or adjustments to the data that are not explicitly stated in the paper. Replicating academic research often involves making assumptions or approximations when exact methodologies are not available or certain details are not provided. These assumptions could differ from those made by the original authors and contribute to differences in results. Our excess returns on stocks were around 4 basis points off, and this leads to all the other portfolios being slightly off due to them using these excess returns numbers.

Lastly, some of the t-stat measurements were more than 10 basis points off. This would follow from above because of dicrepancies in the excess return replication. Therefore the excess return replication would have to be improved first to improve the t-stat.

In [28]:
def annualized_portfolio_returns(df):

    Stock_Excess_Vw_mean = 1200*(df['Stock_Excess_Vw_Ret']).mean()
    Stock_Excess_Vw_std = np.sqrt(12)*100*(df['Stock_Excess_Vw_Ret']).std()
    Stock_Excess_Vw_t_stat = np.sqrt(len(df))*(df['Stock_Excess_Vw_Ret']).mean()/(df['Stock_Excess_Vw_Ret']).std()
    Stock_Excess_Vw_SR = Stock_Excess_Vw_mean/Stock_Excess_Vw_std
    Stock_Excess_Vw_Skew = df['Stock_Excess_Vw_Ret'].skew()
    Stock_Excess_Vw_Kurtosis = df['Stock_Excess_Vw_Ret'].kurtosis()

    Bond_Excess_Vw_mean = 1200*(df['Bond_Excess_Vw_Ret']).mean()
    Bond_Excess_Vw_std = np.sqrt(12)*100*(df['Bond_Excess_Vw_Ret']).std()
    Bond_Excess_Vw_t_stat = np.sqrt(len(df))*(df['Bond_Excess_Vw_Ret']).mean()/(df['Bond_Excess_Vw_Ret']).std()
    Bond_Excess_Vw_SR = Bond_Excess_Vw_mean/Bond_Excess_Vw_std
    Bond_Excess_Vw_Skew = df['Bond_Excess_Vw_Ret'].skew()
    Bond_Excess_Vw_Kurtosis = df['Bond_Excess_Vw_Ret'].kurtosis()
    
    Excess_Vw_mean = 1200*(df['Excess_Vw_Ret']).mean()
    Excess_Vw_std = np.sqrt(12)*100*(df['Excess_Vw_Ret']).std()
    Excess_Vw_t_stat = np.sqrt(len(df))*(df['Excess_Vw_Ret']).mean()/(df['Excess_Vw_Ret']).std()
    Excess_Vw_SR = Excess_Vw_mean/Excess_Vw_std
    Excess_Vw_Skew = df['Excess_Vw_Ret'].skew()
    Excess_Vw_Kurtosis = df['Excess_Vw_Ret'].kurtosis()

    Excess_60_40_mean = 1200*(df['Excess_60_40_Ret']).mean()
    Excess_60_40_std = np.sqrt(12)*100*(df['Excess_60_40_Ret']).std()
    Excess_60_40_t_stat = np.sqrt(len(df))*(df['Excess_60_40_Ret']).mean()/(df['Excess_60_40_Ret']).std()
    Excess_60_40_SR = Excess_60_40_mean/Excess_60_40_std
    Excess_60_40_Skew = df['Excess_60_40_Ret'].skew()
    Excess_60_40_Kurtosis = df['Excess_60_40_Ret'].kurtosis()

    Excess_Unlevered_RP_mean = 1200*(df['Excess_Unlevered_RP_Ret']).mean()
    Excess_Unlevered_RP_std = np.sqrt(12)*100*(df['Excess_Unlevered_RP_Ret']).std()
    Excess_Unlevered_RP_t_stat = np.sqrt(len(df))*(df['Excess_Unlevered_RP_Ret']).mean()/(df['Excess_Unlevered_RP_Ret']).std()
    Excess_Unlevered_RP_SR = Excess_Unlevered_RP_mean/Excess_Unlevered_RP_std
    Excess_Unlevered_RP_Skew = (df['Excess_Unlevered_RP_Ret']).skew()
    Excess_Unlevered_RP_Kurtosis = (df['Excess_Unlevered_RP_Ret']).kurtosis()

    Excess_Levered_RP_mean = 1200*(df['Excess_Levered_RP_Ret']).mean()
    Excess_Levered_RP_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret']).std()
    Excess_Levered_RP_t_stat = np.sqrt(len(df))*(df['Excess_Levered_RP_Ret']).mean()/(df['Excess_Levered_RP_Ret']).std()
    Excess_Levered_RP_SR = Excess_Levered_RP_mean/Excess_Levered_RP_std
    Excess_Levered_RP_Skew = (df['Excess_Levered_RP_Ret']).skew()
    Excess_Levered_RP_Kurtosis = (df['Excess_Levered_RP_Ret']).kurtosis()

    RP_minus_Vw_mean = 1200*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).mean()
    RP_minus_Vw_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).std()
    RP_minus_Vw_t_stat = np.sqrt(len(df))*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).mean()/(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).std()
    RP_minus_Vw_SR = RP_minus_Vw_mean/RP_minus_Vw_std
    RP_minus_Vw_Skew = (df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).skew()
    RP_minus_Vw_Kurtosis = (df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).kurtosis()

    RP_minus_60_40_mean = 1200*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).mean()
    RP_minus_60_40_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).std()
    RP_minus_60_40_t_stat = np.sqrt(len(df))*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).mean()/(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).std()
    RP_minus_60_40_SR = RP_minus_60_40_mean/RP_minus_60_40_std
    RP_minus_60_40_Skew = (df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).skew()
    RP_minus_60_40_Kurtosis = (df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).kurtosis()

    data = {
        'Index': ['CRSP stocks', 'CRSP bonds', 'Value-weighted portfolio', '60/40 portfolio', 'RP, unlevered', 'RP', 'RP minus value-weighted', 'RP minus 60/40'],
        'Excess Return': [Stock_Excess_Vw_mean, Bond_Excess_Vw_mean, Excess_Vw_mean, Excess_60_40_mean, Excess_Unlevered_RP_mean, Excess_Levered_RP_mean, RP_minus_Vw_mean, RP_minus_60_40_mean],
        't-stat': [Stock_Excess_Vw_t_stat, Bond_Excess_Vw_t_stat, Excess_Vw_t_stat, Excess_60_40_t_stat, Excess_Unlevered_RP_t_stat, Excess_Levered_RP_t_stat, RP_minus_Vw_t_stat, RP_minus_60_40_t_stat],
        'Volatility': [Stock_Excess_Vw_std, Bond_Excess_Vw_std, Excess_Vw_std, Excess_60_40_std, Excess_Unlevered_RP_std, Excess_Levered_RP_std, RP_minus_Vw_std, RP_minus_60_40_std],
        'Sharpe Ratio': [Stock_Excess_Vw_SR, Bond_Excess_Vw_SR, Excess_Vw_SR, Excess_60_40_SR, Excess_Unlevered_RP_SR, Excess_Levered_RP_SR, RP_minus_Vw_SR, RP_minus_60_40_SR],
        'Skewness': [Stock_Excess_Vw_Skew, Bond_Excess_Vw_Skew, Excess_Vw_Skew, Excess_60_40_Skew, Excess_Unlevered_RP_Skew, Excess_Levered_RP_Skew, RP_minus_Vw_Skew, RP_minus_60_40_Skew],
        'Kurtosis': [Stock_Excess_Vw_Kurtosis, Bond_Excess_Vw_Kurtosis, Excess_Vw_Kurtosis, Excess_60_40_Kurtosis, Excess_Unlevered_RP_Kurtosis, Excess_Levered_RP_Kurtosis, RP_minus_Vw_Kurtosis, RP_minus_60_40_Kurtosis]
    }
    
    df3 = pd.DataFrame(data)
    return(df3.round(2))

In [29]:
a2 = a2[(a2['Year'] >= 1929) & ((a2['Year'] < 2010) | ((a2['Year'] == 2010) & (a2['Month'] <= 6)))].copy()
a3 = annualized_portfolio_returns(a2)
print("    Portfolio Replication Table: January 1929 to June 2010")
a3

    Portfolio Replication Table: January 1929 to June 2010


,Index,Excess Return,t-stat,Volatility,Sharpe Ratio,Skewness,Kurtosis
0,CRSP stocks,6.75,3.19,19.08,0.35,0.23,7.68
1,CRSP bonds,1.40,4.32,2.93,0.48,0.23,4.19
2,Value-weighted portfolio,3.53,2.65,12.03,0.29,-0.53,4.50
3,60/40 portfolio,4.61,3.57,11.65,0.40,0.24,7.42
4,"RP, unlevered",2.10,5.09,3.73,0.56,0.09,2.64
5,RP,6.68,5.02,12.01,0.56,-0.40,1.98
6,RP minus value-weighted,3.15,3.07,9.24,0.34,0.07,3.61
7,RP minus 60/40,2.07,2.12,8.81,0.23,-0.71,6.47
